# 🎧 Podcast Listening Time Prediction
_A data science project to predict [podcast listening duration](https://www.kaggle.com/competitions/playground-series-s5e4/overview)_

---

### **Created by: Antonio Kevin**
🌐 [**Kaggle**](https://www.kaggle.com/akkevin) | 💼 [**LinkedIn**](https://www.linkedin.com/in/antonio-kevin/) | 🧑‍💻 [**GitHub**](https://github.com/akkevinn)

---

## **Table of Contents**
1. [Problem Understanding](#Problem-Understanding)
2. [Approach](#Approach)
3. [Data Preprocessing](#Data-Preprocessing)
4. [Feature Engineering](#Feature-Engineering)
5. [Model Training](#Model-Training)
6. [Prediction & Submission](#Prediction-and-Submission)

---

## **Problem Understanding**

### **Business Objective**:
The objective of this model is to **predict podcast listening durations** to help content creators improve their podcasts and increase engagement. This prediction will assist with:

- **Optimizing episode length**: Helping creators decide on the ideal episode duration.
- **Improving publishing schedules**: Suggesting the best times and days for publishing.
- **Guest selection**: Determining which types of guests or topics result in higher engagement.

### **Evaluation Metric**:
The model's performance will be evaluated using **Root Mean Squared Error (RMSE)**, which measures the difference between the predicted and actual listening durations in **minutes**. The goal is to minimize this error to ensure more accurate predictions.

---

## **Approach**
This notebook will walk through the following steps:

1. **Data Preprocessing**: Clean and prepare the data for modeling.
2. **Feature Engineering**: Create meaningful features for model training.
3. **Model Training**: Train and fine-tune a suitable model for the prediction task.
4. **Evaluation**: Assess model performance using RMSE.
5. **Prediction on Test Data and Submission**:
    - Use the trained model to make predictions on the test dataset.
    - Format the predictions for submission according to the competition requirements.
    - Save the results to a CSV file for submission.

---

In [1]:
!pip install category_encoders

from google.colab import drive
import pandas as pd
import numpy as np
from category_encoders import TargetEncoder
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.metrics import mean_squared_error

import sklearn
sklearn.set_config(transform_output="pandas")

# Mount the drive to access files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Data Preprocessing**

In [2]:
# Load data
file_path = '/content/drive/My Drive/Google Colab/podcast_listening_time/train.csv'
train_data = pd.read_csv(file_path)

file_path = '/content/drive/My Drive/Google Colab/podcast_listening_time/test.csv'
test_data = pd.read_csv(file_path)

print(f"Train shape: {train_data.shape}, Test shape: {test_data.shape}")

Train shape: (750000, 12), Test shape: (250000, 11)


In [3]:
# Import dataset
train_data['dataset'] = 'train'
test_data['dataset'] = 'test'

# Combine both train & test data for feature engineering
data = pd.concat([train_data, test_data]).reset_index(drop=True)
data.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,dataset
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998,train
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241,train
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531,train
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824,train
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031,train


In [4]:
# Number of Ads
## Fill missing (NULL) values with the median of 'Number_of_Ads' column
## Censor outlier values by setting a maximum threshold of 3 for any values greater than 3
data['Number_of_Ads'] = data['Number_of_Ads'].fillna(data['Number_of_Ads'].median())
data['Number_of_Ads'] = data['Number_of_Ads'].clip(upper=3)

# Guest Popularity Percentage
## Fill missing (NULL) values with 0, assuming no guest in the podcast if the value is missing
## Censor outlier values by setting a maximum threshold of 100 for any values greater than 100
data['Guest_Popularity_percentage'] = data['Guest_Popularity_percentage'].fillna(0)
data['Guest_Popularity_percentage'] = data['Guest_Popularity_percentage'].clip(upper=100)

# Episode Length Minutes
## Fill missing (NULL) values in the 'Episode_Length_minutes' column with the median value for each podcast
## Censor outlier values by setting a maximum threshold of 120 minutes for any values greater than 120
data['Episode_Length_minutes'] = \
    data.groupby('Podcast_Name')['Episode_Length_minutes'] \
    .transform(lambda x: x.fillna(x.median()))
data['Episode_Length_minutes'] = data['Episode_Length_minutes'].clip(upper=120)

# Host Popularity Percentage
## Censor outlier values by setting a maximum threshold of 100 for any values greater than 100
data['Host_Popularity_percentage'] = data['Host_Popularity_percentage'].clip(upper=100)

# Create episode number feature
## Extract episode number from the 'Episode_Title' column (assumes episode number is in the title)
## Convert the extracted value to a float and fill any missing episode numbers with 0
data['episode_num'] = data['Episode_Title'].str.extract('(\d+)').astype(float).fillna(0)

## **Feature Engineering**

In [5]:
# Calculate Average of Episode Length Minute per each category
category_list = ['Episode_Sentiment', 'Genre', 'Publication_Day', 'Publication_Time', 'Podcast_Name', 'Number_of_Ads']
for col in category_list:
  data['avg_time_by_' + col] = data.groupby(col)['Episode_Length_minutes'].transform('mean')

# Genre
## Apply One-Hot Encoding to the 'Genre' column
## This converts each unique genre into a separate binary column (1 or 0)
data = pd.get_dummies(data, columns=['Genre'], prefix='genre')

# Publication Day
## Use sine and cosine functions to encode the days of the week cyclically
## These columns will represent the periodic nature of the 'Publication_Day' feature
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday',
             'Friday', 'Saturday', 'Sunday']
data['day_num'] = data['Publication_Day'].map(lambda x: day_order.index(x))
data['day_sin'] = np.sin(2 * np.pi * data['day_num'] / 7)
data['day_cos'] = np.cos(2 * np.pi * data['day_num'] / 7)

# Publication Time
## Apply One-Hot Encoding to the 'Publication_Time' column
## This will convert each unique publication time into separate binary columns
data = pd.get_dummies(data, columns=['Publication_Time'], prefix='time')

# Sentiment
## Apply Ordinal Encoding to the 'Episode_Sentiment' column
## Map 'Positive' to 2, 'Neutral' to 1, and 'Negative' to 0 for sentiment analysis
sentiment_map = {'Positive': 2, 'Neutral': 1, 'Negative': 0}
data['sentiment_encoded'] = data['Episode_Sentiment'].map(sentiment_map)

# Create length_minute_bin
## Bin the 'Episode_Length_minutes' column into discrete categories based on episode length
## The bins are set as [0, 30, 60, 90, 120] minutes, and the corresponding labels are [0, 1, 2, 3]
bins = [0, 30, 60, 90, 120]
labels = [0, 1, 2, 3]
data['length_bin'] = pd.cut(data['Episode_Length_minutes'], bins=bins, labels=labels, right=True)

## If 'Episode_Length_minutes' is 0, set the bin to 0
data['length_bin'] = data.apply(lambda x: 0 if x['Episode_Length_minutes'] == 0 else x['length_bin'], axis=1)

## Convert the 'length_bin' to an integer type
data['length_bin'] = data['length_bin'].astype(int)

# Create Is Weekend
## Create a binary column indicating whether the publication day is a weekend (Saturday or Sunday)
data['is_weekend'] = data['Publication_Day'].isin(['Saturday', 'Sunday']).astype(int)

# Compare Host & Guest Popularity
## Calculate the ratio of host popularity to guest popularity, with a small epsilon to avoid division by zero
data['host_guest_ratio'] = data['Host_Popularity_percentage'] / (data['Guest_Popularity_percentage'] + 1e-6)

## Calculate the difference between host and guest popularity
data['host_guest_diff'] = data['Host_Popularity_percentage'] - data['Guest_Popularity_percentage']

# Amplify popularity metrics based on sentiment
## Multiply the host popularity by the sentiment encoding to adjust for sentiment impact
data['host_pop_sentiment'] = data['Host_Popularity_percentage'] * data['sentiment_encoded']

## Multiply the guest popularity by the sentiment encoding to adjust for sentiment impact
data['guest_pop_sentiment'] = data['Guest_Popularity_percentage'] * data['sentiment_encoded']

# Create flag for high popularity
data['is_high_host_popularity'] = (data['Host_Popularity_percentage'] > 60).astype(int)
data['is_high_guest_popularity'] = (data['Guest_Popularity_percentage'] > 60).astype(int)

# Create ads per minute
data['ad_per_minute'] = data['Number_of_Ads'] / ( data['Episode_Length_minutes'] + 1e-6)

# Create flag for long episode
data['is_long_episode'] = (data['Episode_Length_minutes'] > 60).astype(int)

In [7]:
# Preparing for modeling
features = [
    'day_sin', 'day_cos', 'sentiment_encoded',
    'episode_num', 'Episode_Length_minutes',
    'Host_Popularity_percentage', 'Guest_Popularity_percentage',
    'Number_of_Ads',
    'length_bin',
    'is_weekend',
    'host_guest_ratio',
    'host_guest_diff',
    'host_pop_sentiment',
    'guest_pop_sentiment',
    'is_high_host_popularity',
    'is_high_guest_popularity',
    'ad_per_minute',
    'is_long_episode',
    'Podcast_Name'
] + [col for col in data.columns if col.startswith('time_')] \
    + [col for col in data.columns if col.startswith('genre_')] \
    + [col for col in data.columns if col.startswith('avg_time_by_')]
features

['day_sin',
 'day_cos',
 'sentiment_encoded',
 'episode_num',
 'Episode_Length_minutes',
 'Host_Popularity_percentage',
 'Guest_Popularity_percentage',
 'Number_of_Ads',
 'length_bin',
 'is_weekend',
 'host_guest_ratio',
 'host_guest_diff',
 'host_pop_sentiment',
 'guest_pop_sentiment',
 'is_high_host_popularity',
 'is_high_guest_popularity',
 'ad_per_minute',
 'is_long_episode',
 'Podcast_Name',
 'time_Afternoon',
 'time_Evening',
 'time_Morning',
 'time_Night',
 'genre_Business',
 'genre_Comedy',
 'genre_Education',
 'genre_Health',
 'genre_Lifestyle',
 'genre_Music',
 'genre_News',
 'genre_Sports',
 'genre_Technology',
 'genre_True Crime',
 'avg_time_by_Episode_Sentiment',
 'avg_time_by_Genre',
 'avg_time_by_Publication_Day',
 'avg_time_by_Publication_Time',
 'avg_time_by_Podcast_Name',
 'avg_time_by_Number_of_Ads']

In [8]:
# Split train & test data
train_data = data[data['dataset'] == 'train'].copy()
test_data = data[data['dataset'] == 'test'].reset_index(drop=True).copy()

## **Model Training**

In [9]:
# Prepare features and target variable
X = train_data[features]
y = train_data['Listening_Time_minutes']

# Prepare test data
X_test = test_data[features]

In [9]:
# Training model
kf = KFold(n_splits=5)
rmse_scores = []

for (train_idx, val_idx) in kf.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # Initialize target encoder
    encoder = TargetEncoder(cols=['Podcast_Name'], smoothing=10)

    # Fit and transform on training fold
    X_train_encoded = encoder.fit_transform(X_train, y_train)

    # Transform validation fold (using training fold's encoding)
    X_val_encoded = encoder.transform(X_val)

    # Drop original Podcast Name column
    X_train_encoded = X_train_encoded.drop(columns=['Podcast_Name'])
    X_val_encoded = X_val_encoded.drop(columns=['Podcast_Name'])

    # Train model
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=2000,
        learning_rate=0.02,
        max_depth=15,
        subsample=0.8,
        early_stopping_rounds=50,
        eval_metric='rmse',
        tree_method='hist',
        device='cuda'
    )
    model.fit(X_train_encoded, y_train, eval_set=[(X_val_encoded, y_val)], verbose=False)

    # Validate
    val_preds = model.predict(X_val_encoded)
    rmse = np.sqrt(mean_squared_error(y_val, val_preds))
    rmse_scores.append(rmse)
    print(f"Fold RMSE: {rmse:.2f} minutes")

print(f"\nAverage RMSE: {np.mean(rmse_scores):.2f} ± {np.std(rmse_scores):.2f}")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:21:31] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Fold RMSE: 12.69 minutes
Fold RMSE: 12.67 minutes
Fold RMSE: 12.60 minutes
Fold RMSE: 12.60 minutes
Fold RMSE: 12.61 minutes

Average RMSE: 12.63 ± 0.04


## **Prediction and Submission**

In [10]:
# Final model training (RMSE: 12.57005)
encoder = TargetEncoder(cols=['Podcast_Name'], smoothing=10)
X_encoded = encoder.fit_transform(X, y)
X_encoded = X_encoded.drop(columns=['Podcast_Name'])
X_test_encoded = encoder.transform(X_test).drop(columns=['Podcast_Name'])

final_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=2000,
    learning_rate=0.02,
    max_depth=15,
    subsample=0.8,
    tree_method='hist',
    device='cuda'
)
final_model.fit(X_encoded, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.02, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=15, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=2000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [11]:
# Prepare submission
test_preds = final_model.predict(X_test_encoded)
submission_data = pd.DataFrame({
    'id': test_data['id'],
    'Listening_Time_minutes': test_preds
})
submission_data.to_csv('/content/drive/My Drive/Colab Notebooks/Podcast Listening Time/final_submission_v15.csv', index=False)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:55:19] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
